In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy import interpolate
from scipy.integrate import simps, romb, trapz

In [2]:
def findexp_replace(folder,filename):
    logtmp=os.path.isfile("tmp")
    if logtmp==True:
        print(filename+" converted")
        return
    print("converting "+filename)
    os.system("cp "+folder+filename+" tmp")
    os.system("sed -i 's/ /  /g' tmp")
    os.system("sed -i 's/+/E+/g' tmp")
    os.system("sed -i 's/-/E-/g' tmp")
    os.system("sed -i 's/(/,/g' tmp")
    os.system("sed -i 's/)/,/g' tmp")
    for i in range(99):
        if i<10: os.system("sed -i 's/+0"+str(i)+"E/+0"+str(i)+" /g' tmp")
        if i<10: os.system("sed -i 's/-0"+str(i)+"E/-0"+str(i)+" /g' tmp")
        if i>=10: os.system("sed -i 's/+"+str(i)+"E/+"+str(i)+" /g' tmp")
        if i>=10: os.system("sed -i 's/-"+str(i)+"E/-"+str(i)+" /g' tmp")

In [3]:
def interp_omg(n,x,y,xnew,kind):
    if kind=="cubic": spline=interpolate.interp1d(x, y, kind=kind)
    if kind=="akima": spline=interpolate.Akima1DInterpolator(x,y)
    ynew=spline(xnew)
    return ynew

In [4]:
def nconfig(name):
    ncfg=0
    if (name=='2p6.3s2') or (name=='3S2'): ncfg=1
    if (name=='3s.3p') or (name=='3S1 3P1'): ncfg=2
    if (name=='3s.3d') or (name=='3S1 3D1'): ncfg=3
    if (name=='3s.4s') or (name=='3S1 4S1'): ncfg=4
    if (name=='3s.4p') or (name=='3S1 4P1'): ncfg=5
    if (name=='3s.4d') or (name=='3S1 4D1'): ncfg=6
    if (name=='3s.4f') or (name=='3S1 4F1'): ncfg=7
    if (name=='3s.5s') or (name=='3S1 5S1'): ncfg=8
    if (name=='3s.5p') or (name=='3S1 5P1'): ncfg=9
    if (name=='3s.5d') or (name=='3S1 5D1'): ncfg=10
    if (name=='3s.5f') or (name=='3S1 5F1'): ncfg=11
    if (name=='3s.5g') or (name=='3S1 5G1'): ncfg=12
    if (name=='3p2') or (name=='3P2'): ncfg=13
    if (name=='3s.6s') or (name=='3S1 6S1'): ncfg=14
    if (name=='3s.6p') or (name=='3S1 6P1'): ncfg=15
    if (name=='3s.6d') or (name=='3S1 6D1'): ncfg=16
    if (name=='3s.6f') or (name=='3S1 6F1'): ncfg=17
    if (name=='3s.6g') or (name=='3S1 6G1'): ncfg=18
    if (name=='3s.6h') or (name=='3S1 6H1'): ncfg=19
    if (name=='3s.7s') or (name=='3S1 7S1'): ncfg=20
    if (name=='3s.7p') or (name=='3S1 7P1'): ncfg=21
    if (name=='3s.7d') or (name=='3S1 7D1'): ncfg=22
    if (name=='3s.7f') or (name=='3S1 7F1'): ncfg=23
    if (name=='3s.7g') or (name=='3S1 7G1'): ncfg=24
    if (name=='3s.7h') or (name=='3S1 7H1'): ncfg=25
    if (name=='3s.7i') or (name=='3S1 7I1'): ncfg=26
    if (name=='3s.8s') or (name=='3S1 8S1'): ncfg=27
    if (name=='3s.8p') or (name=='3S1 8P1'): ncfg=28
    if (name=='3s.9s') or (name=='3S1 9S1'): ncfg=29    
    return ncfg

In [5]:
def map_BTspace(xener,eik,yomg,ntype):
    C=np.e
    xx=xener/eik
    if ntype==1:
        x=1-np.log(C)/np.log(xx+C)
        y=yomg/np.log(xx+np.e)
    if ntype==2:
        x=xx/(xx+C)
        y=yomg
    if ntype==3:
        x=xx/(xx+C)
        y=(xx+1)**2*yomg
    if ntype==4:
        x=1-np.log(C)/np.log(xx+C)
        y=yomg/np.log(xx+C)
    # include first point in BT-space
    lx=list(x)
    lx.insert(0,0)
    x=np.array(lx)
    ly=list(y)
    ly.insert(0,0)
    y=np.array(ly)
    # include last point in BT-space
    lx=list(x)
    xlast=0.99
#     if ntype==1: xlast=0.99
#     if ntype==3: xlast=0.99
    if lx[-1]>xlast: xlast=abs(lx[-1]-1.)/2.+lx[-1]
    lx.append(xlast)
    x=np.array(lx)
    if ntype==1 or ntype==4: ylast=abs(omg.loc[j][-2])
    if ntype==2 or ntype==3: ylast=y[-2]+(xlast-x[-2])/(x[-1]-x[-2])*(y[-1]-y[-2])
    ly=list(y)
    ly.append(ylast)
    y=np.array(ly)
    return x,y

In [6]:
def mapback_BTspace(ntype,eik,x_new,y_new):
    C=np.e
    if ntype==1:
        arg=np.log(C)/(1.-x_new)
        enerp=(np.exp(arg)-C)*eik
        omgp=y_new*np.log(enerp/eik+np.e)
    if ntype==2:
        enerp=x_new/(1.-x_new)*C*eik
        omgp=y_new
    if ntype==3:
        enerp=x_new*C/(1.-x_new)*eik
        omgp=y_new/(enerp/eik+1)**2
    if ntype==4:
        arg=np.log(C)/(1.-x_new)
        enerp=(np.exp(arg)-C)*eik
        omgp=y_new*np.log(enerp/eik+C)
    omgp[omgp < 0.] = 0.
#     nmax=npts-1
    nmax=len([i for i in enerp if i < 10.])
    enerpp=enerp[:nmax]
    omgpp=omgp[:nmax]
    return enerpp,omgpp

In [7]:
def data_trans(j,omg,levels):
    k=int(omg.loc[j]['k'])
    i=int(omg.loc[j]['i'])
    Aki=omg.loc[j]['aki']
    jk=levels.loc[k]['J']
    ji=levels.loc[i]['J']
    gk=2*jk+1
    gi=2*ji+1
    ek=levels.loc[k]['E']
    ei=levels.loc[i]['E']
    eik=abs(ek-ei)
    fik=0
    if eik!=0: 
        S=3.73491E-10*gk*Aki/eik**3
        fik=eik*S/(3.*gi)
    return Aki,gk,gi,eik,fik

In [8]:
def compute_ECS(enerpp,omgpp,ups_new):
    global ntemp,T,kB,convRyd2eV
    yups=[]
    for i in range(ntemp):
        kBT=kB*T[i]
        arg=enerpp*convRyd2eV/kBT
        yint=omgpp*np.exp(-arg)
        I1=simps(yint, arg)
        yups.append(I1)
        ups_new.loc[j,str(T[i])]=I1
    return yups

In [9]:
def plot_om2ups(xener,yomg,enerpp,omgpp,x,y,x_new,y_new,T,ups):
    fig=plt.figure(figsize=(15,5))
    ax1=plt.subplot(131)
    plt.plot(xener,yomg,'o-')
    plt.plot(enerpp,omgpp,'r-')
    ax2=plt.subplot(132)
    plt.plot(x,y,'o-')
    plt.plot(x_new,y_new,'r-')
    ax3=plt.subplot(133)
#     plt.plot(T,ups_prev.loc[0][3:-1],'k',label="adas")
    plt.plot(T,yups,'g-',label="present")
    plt.legend(loc='upper left',bbox_to_anchor=[1,1],fontsize=15,ncol=1)
    plt.show()

# Define final scattered electron energy and temperatures

In [10]:
global ntemp,T,kB,convRyd2eV,npts,ntran

In [11]:
kB=8.6173324E-05 # eV/K
convRyd2eV=13.6057 # eV/Ryd
convcm2Ryd=1./109737.26 

In [12]:
ener=[1.00E-02,2.20E-01,4.89E-01,8.17E-01,1.22E+00,
      1.71E+00,2.30E+00,3.03E+00,3.92E+00,5.00E+00]
T=[3.0E+03,4.0E+03,5.0E+03,6.0E+03,7.0E+03,8.0E+03,9.0E+03,
   1.0E+04,1.1E+04,1.2E+04,1.3E+04,1.4E+04,1.5E+04,1.6E+04]

In [13]:
nener=len(ener)
ntemp=len(T)
ntran=3175
nlevels=90
npts=2**13+1

In [14]:
folder="./"
file_om="adf04_om"
file_ups="adf04_ups"

# Load level data and transitions collision strength:

In [15]:
findexp_replace(folder,file_om)

In [16]:
levels=pd.read_csv("tmp",header=None,skiprows=[0],nrows=nlevels,usecols=[0,1,2,3,4])
colsname=['CFG','S','L','J','E']
levels.columns=colsname
for j in range(nlevels):
    s=levels.loc[j]['CFG']
    sn=" ".join(s.split()[1:])
    levels.at[j,'CFG']=sn
levels.index=levels.index+1
levels['E']=levels['E']*convcm2Ryd

In [17]:
omgcols=[i for i in range(nener+4)]
omg=pd.read_csv("tmp",sep="\s+",header=None,skiprows=93,nrows=ntran,usecols=omgcols)
colsname= ['k','i','aki']
for i in range(nener):
    colsname.append(str(ener[i]))
colsname.append("inf")
omg.columns=colsname

In [18]:
levels.head()

,CFG,S,L,J,E
1,3S2,1,0,0.0,0.000000
2,3S1 3P1,3,1,4.0,0.173051
3,3S1 3P1,1,1,1.0,0.336431
4,3S1 4S1,3,0,1.0,0.346630
5,3S1 4S1,1,0,0.0,0.383369


In [19]:
omg.head()

,k,i,aki,0.01,0.22,0.489,0.817,1.22,1.71,2.3,3.03,3.92,5.0,inf
0,2,1,1.000000e-30,1.52,6.45,4.670,2.450,1.1600,0.5780,0.3140,0.1870,0.11500,0.07240,0.0
1,3,1,4.280000e+08,3.14,8.18,14.900,22.000,28.8000,34.6000,40.4000,45.6000,50.60000,55.50000,-19.6
2,3,2,1.000000e-30,11.90,11.80,4.350,1.770,0.7680,0.3700,0.1970,0.1140,0.06830,0.04190,0.0
3,4,1,1.000000e-30,1.06,1.18,0.385,0.199,0.0892,0.0409,0.0216,0.0128,0.00795,0.00501,0.0
4,4,2,9.750000e+07,11.80,27.10,29.600,43.700,41.5000,59.5000,55.2000,63.5000,72.00000,80.40000,-26.8


# Sort type of transition

In [20]:
fbig=0.01
fzero=1.0E-04

In [21]:
ntype=[]
for j in range(ntran):
    k=int(omg.loc[j]['k'])
    i=int(omg.loc[j]['i'])
    Aki=omg.loc[j]['aki']
    jk=levels.loc[k]['J']
    ji=levels.loc[i]['J']
    gk=2*jk+1
    gi=2*ji+1
    ek=levels.loc[k]['E']
    ei=levels.loc[i]['E']
    eik=abs(ek-ei)
    if eik==0: eik=0.00001
    S=3.73491E-10*gk*Aki/eik**3
    fij=eik*S/(3.*gi)
    if levels.loc[k]['S']==levels.loc[i]['S']:
        if (abs(levels.loc[k]['L']-levels.loc[i]['L'])<=1) & (fij>=fbig):
            ntype.append(1)
        else:
            if (fij>fzero) & (fij<fbig):
                ntype.append(4)
            else:
                ntype.append(2)
    else:
        if (fij>fzero) & (fij<fbig):
            ntype.append(4)
        else:
            ntype.append(3)
        #ntype.append(3)
omg['type']=ntype

In [22]:
omg.head()

,k,i,aki,0.01,0.22,0.489,0.817,1.22,1.71,2.3,3.03,3.92,5.0,inf,type
0,2,1,1.000000e-30,1.52,6.45,4.670,2.450,1.1600,0.5780,0.3140,0.1870,0.11500,0.07240,0.0,3
1,3,1,4.280000e+08,3.14,8.18,14.900,22.000,28.8000,34.6000,40.4000,45.6000,50.60000,55.50000,-19.6,1
2,3,2,1.000000e-30,11.90,11.80,4.350,1.770,0.7680,0.3700,0.1970,0.1140,0.06830,0.04190,0.0,3
3,4,1,1.000000e-30,1.06,1.18,0.385,0.199,0.0892,0.0409,0.0216,0.0128,0.00795,0.00501,0.0,3
4,4,2,9.750000e+07,11.80,27.10,29.600,43.700,41.5000,59.5000,55.2000,63.5000,72.00000,80.40000,-26.8,1


# Insert database info

In [23]:
mod_levels=pd.read_csv("ModelAtomicIonLevel.txt",sep="\s+",header='infer',nrows=55,usecols=[3,6,7,8,9,11,12])
mod_levels.rename(columns={'2S':'S'},inplace=True)
mod_levels['S']=mod_levels['S']+1
mod_levels['ExcitationWaven']=mod_levels['ExcitationWaven']*convcm2Ryd
nmodlev=len(mod_levels.index)

In [24]:
mod_levels.tail()

,LevelNumber,ElectronConfig,Term,S,L,LevelWeight,ExcitationWaven
50,51,3s.7h,3H*,3,5,33,0.541556
51,52,3s.7h,1H*,1,5,11,0.541556
52,53,3s.7i,3I,3,6,39,0.541571
53,54,3s.7i,1I,1,6,13,0.541571
54,55,3s.9s,3S,3,0,3,0.543563


# Burgess and Tully interpolation + effective collision strength calculation:

In [25]:
ups=omg.copy()
enercols=[str(ener[i]) for i in range(nener)]
tmpcols=[str(T[i]) for i in range(ntemp)]
enercols.append("type")
ups.drop(enercols, axis=1, inplace=True)
tmp=[0 for i in range(ntran)]
for i in range(ntemp):
    ii=i+3
    ups.insert(ii,tmpcols[i], tmp)

In [26]:
ups.head()

,k,i,aki,3000.0,4000.0,5000.0,6000.0,7000.0,8000.0,9000.0,10000.0,11000.0,12000.0,13000.0,14000.0,15000.0,16000.0,inf
0,2,1,1.000000e-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,3,1,4.280000e+08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-19.6
2,3,2,1.000000e-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,4,1,1.000000e-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,4,2,9.750000e+07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-26.8


# Change type of some transition for better mapping

In [27]:
ntype2=[34,77,132]
for i in ntype2:
    omg.at[i,'type']=2

In [28]:
ntype4=[5,9,19,20,25,26,35,44,45,54,64,76,88,89,90,
        101,102,117,118,119,133,134,151,152,170,
        208,209,228,229,230,249,250,271,272,249,
        296,297,298,299,320,321,322,323,344,345,
        345,346,347,376,404,462,463,492,493,494,
        523,524,525,555,556,588,589,592,624,625,
        626,627,628,659,660,661,662,663,694,695,
        696,697,698,699,730,731,732,733,734,766,
        773,774,775,776,816,846,845,846,847,848,
        849,850,851,856,893,894,895,896,898,899,
        935,936,937,938,940,942,983,984,
        1015,1016,1022,1021,1022,1023,1024,1025,1027,1028,
        1067,1068,1069,1070,1072,1073,1075,1076,1113,1114,
        1115,1116,1118,1119,1120,1121,1122,1123,1155,1159,
        1164,1167,1168,1169,1170,1171,1172,1203,1204,1205,
        1206,1207,1212,1215,1216,1217,1218,1219,1220,1251,
        1252,1253,1254,1255,1260,1263,1264,1265,1266,1267,
        1268,1269,1300,1301,1302,1303,1304,1309,1312,1313,
        1314,1315,1316,1317,1318,1319,1363,1364,1365,1366,
        1367,1408,1412,1413,1420,1421,1422,1423,1424,1469,
        1470,1471,1472,1477,1478,1518,1521,1522,1523,1524,
        1525,1526,1532,1582,1583,1584,1585,1586,1588,1589,
        1626,1631,1638,1639,1640,1641,1642,1644,1645,1688,
        1695,1696,1697,1698,1699,1701,1702,1746,1753,1754,
        1755,1756,1757,1759,1760,1804,1805,1812,1813,1814,
        1815,1816,1818,1819,1822,1854,1855,1856,1857,1858,
        1863,1866,1868,1869,1870,1871,1877,1880,1881,1882,
        1883,1884,1824,1927,1928,1929,1930,1931,1932,1938,
        1941,1942,1943,1944,1945,1976,1977,1978,1979,1980,
        1988,1989,1990,1991,1992,1993,1999,2002,2003,2004,
        2005,2006,2007,2022,2021,2022,2023,2024,2025,2026,
        2027,2028,2029,2030,2073,2074,2075,2076,2077,2078,
        2084,2087,2088,2089,2090,2091,2095,2144,2145,2146,
        2147,2148,2150,2151,2157,2158,2159,2160,2207,2208,
        2214,2217,2218,2219,2220,2221,2225,2227,2284,2285,
        2286,2287,2288,2292,2294,2295,2357,2358,2359,2360,
        2362,2363,2412,2413,2414,2415,2419,2425,2426,2427,
        2429,2431,2432,2482,2483,2484,2485,2486,2488,2495,
        2496,2497,2498,2500,2501,2552,2553,2254,2255,2256,
        2558,2559,2565,2566,2567,2568,2570,2571,2572,2573,
        2610,2613,2615,2616,2623,2624,2625,2627,2629,2630,
        2636,2637,2638,2639,2641,2642,2643,2644,2645,2682,
        2683,2684,2685,2687,2688,2695,2696,2697,2698,2699,
        2701,2702,2708,2709,2710,2711,2713,2714,2715,2716,
        2717,2718,2768,2769,2770,2771,2772,2774,2775,2781,
        2782,2783,2784,2786,2787,2788,2789,2790,2791,2792,
        2836,2837,2838,2839,2840,2841,2847,2850,2851,2852,
        2853,2854,2859,2862,2863,2864,2865,2866,2911,2912,
        2913,2914,2915,2916,2922,2925,2926,2927,2928,2929,
        2934,2937,2938,2939,2940,2941,2986,2987,2988,2990,
        2991,3000,3001,3002,3003,3004,3009,3012,3013,3014,
        3015,3016,3017,3018,3019,3062,3063,3064,3065,3066,
        3067,3073,3076,3077,3078,3079,3080,3085,3088,3089,
        3090,3091,3092,3093,3094,3095,3096,3139,3140,3141,
        3142,3143,3144,3150,3151,3152,3153,3155,3156,3157,
        3162,3165,3166,3167,3168,3169,3170,3171,3172,3173,
        3174]
for i in ntype4:
    omg.at[i,'type']=4

In [29]:
for j in range(ntran):
    ntype=int(omg.loc[j]['type'])
    print("j=",j,"type=",ntype)
    Aki,gk,gi,eik,fik=data_trans(j,omg,levels)
    if eik==0: 
        continue
    # copy data into aux variables
    yomg=omg.loc[j][3:-2].values
    yomg[yomg < 0.] = 0.
    xener=np.array(ener)
    # map into BT-space
    x,y=map_BTspace(xener,eik,yomg,ntype)
    # make spline of x y in BT-space
    nx=len(x)
    x_new=np.linspace(x[0],x[nx-1], npts)
    y_new=interp_omg(nx,x,y,x_new,'akima')
    # mapping back from BT-space
    enerpp,omgpp=mapback_BTspace(ntype,eik,x_new,y_new)
    # compute effective collision strength
    yups=compute_ECS(enerpp,omgpp,ups)
    # plot and check data
    iplot=0
    if iplot==1:
        plot_om2ups(xener,yomg,enerpp,omgpp,x,y,x_new,y_new,T,ups)

j= 0 type= 3
j= 1 type= 1
j= 2 type= 3
j= 3 type= 3
j= 4 type= 1
j= 5 type= 4
j= 6 type= 2
j= 7 type= 3
j= 8 type= 1
j= 9 type= 4
j= 10 type= 2
j= 11 type= 3
j= 12 type= 1
j= 13 type= 3
j= 14 type= 2
j= 15 type= 3
j= 16 type= 2
j= 17 type= 3
j= 18 type= 1
j= 19 type= 4
j= 20 type= 4
j= 21 type= 3
j= 22 type= 1
j= 23 type= 3
j= 24 type= 2
j= 25 type= 4
j= 26 type= 4
j= 27 type= 4
j= 28 type= 1
j= 29 type= 3
j= 30 type= 2
j= 31 type= 3
j= 32 type= 1
j= 33 type= 1
j= 34 type= 2
j= 35 type= 4
j= 36 type= 3
j= 37 type= 1
j= 38 type= 3
j= 39 type= 2
j= 40 type= 3
j= 41 type= 3
j= 42 type= 1
j= 43 type= 2
j= 44 type= 4
j= 45 type= 4
j= 46 type= 3
j= 47 type= 1
j= 48 type= 3
j= 49 type= 2
j= 50 type= 2
j= 51 type= 3
j= 52 type= 3
j= 53 type= 1
j= 54 type= 4
j= 55 type= 2
j= 56 type= 3
j= 57 type= 1
j= 58 type= 3
j= 59 type= 2
j= 60 type= 2
j= 61 type= 3
j= 62 type= 3
j= 63 type= 1
j= 64 type= 4
j= 65 type= 2
j= 66 type= 3
j= 67 type= 1
j= 68 type= 3
j= 69 type= 2
j= 70 type= 3
j= 71 type= 3
j=

j= 557 type= 2
j= 558 type= 2
j= 559 type= 1
j= 560 type= 3
j= 561 type= 2
j= 562 type= 3
j= 563 type= 2
j= 564 type= 3
j= 565 type= 3
j= 566 type= 2
j= 567 type= 2
j= 568 type= 3
j= 569 type= 2
j= 570 type= 3
j= 571 type= 3
j= 572 type= 2
j= 573 type= 2
j= 574 type= 2
j= 575 type= 3
j= 576 type= 3
j= 577 type= 2
j= 578 type= 3
j= 579 type= 3
j= 580 type= 2
j= 581 type= 2
j= 582 type= 2
j= 583 type= 1
j= 584 type= 3
j= 585 type= 3
j= 586 type= 3
j= 587 type= 2
j= 588 type= 4
j= 589 type= 4
j= 590 type= 2
j= 591 type= 2
j= 592 type= 4
j= 593 type= 1
j= 594 type= 2
j= 595 type= 3
j= 596 type= 2
j= 597 type= 3
j= 598 type= 2
j= 599 type= 2
j= 600 type= 3
j= 601 type= 3
j= 602 type= 2
j= 603 type= 3
j= 604 type= 2
j= 605 type= 2
j= 606 type= 3
j= 607 type= 3
j= 608 type= 3
j= 609 type= 2
j= 610 type= 2
j= 611 type= 3
j= 612 type= 2
j= 613 type= 2
j= 614 type= 3
j= 615 type= 3
j= 616 type= 3
j= 617 type= 3
j= 618 type= 1
j= 619 type= 2
j= 620 type= 2
j= 621 type= 3
j= 622 type= 2
j= 623 typ

j= 1099 type= 2
j= 1100 type= 2
j= 1101 type= 2
j= 1102 type= 3
j= 1103 type= 3
j= 1104 type= 3
j= 1105 type= 2
j= 1106 type= 3
j= 1107 type= 3
j= 1108 type= 2
j= 1109 type= 2
j= 1110 type= 2
j= 1111 type= 2
j= 1112 type= 1
j= 1113 type= 4
j= 1114 type= 4
j= 1115 type= 4
j= 1116 type= 4
j= 1117 type= 2
j= 1118 type= 4
j= 1119 type= 4
j= 1120 type= 4
j= 1121 type= 4
j= 1122 type= 4
j= 1123 type= 4
j= 1124 type= 1
j= 1125 type= 2
j= 1126 type= 3
j= 1127 type= 2
j= 1128 type= 3
j= 1129 type= 2
j= 1130 type= 2
j= 1131 type= 3
j= 1132 type= 3
j= 1133 type= 2
j= 1134 type= 3
j= 1135 type= 2
j= 1136 type= 2
j= 1137 type= 3
j= 1138 type= 3
j= 1139 type= 3
j= 1140 type= 2
j= 1141 type= 2
j= 1142 type= 3
j= 1143 type= 2
j= 1144 type= 2
j= 1145 type= 3
j= 1146 type= 3
j= 1147 type= 3
j= 1148 type= 3
j= 1149 type= 2
j= 1150 type= 2
j= 1151 type= 2
j= 1152 type= 3
j= 1153 type= 2
j= 1154 type= 2
j= 1155 type= 4
j= 1156 type= 3
j= 1157 type= 3
j= 1158 type= 3
j= 1159 type= 4
j= 1160 type= 1
j= 1161 

j= 1613 type= 2
j= 1614 type= 3
j= 1615 type= 3
j= 1616 type= 3
j= 1617 type= 1
j= 1618 type= 3
j= 1619 type= 3
j= 1620 type= 4
j= 1621 type= 2
j= 1622 type= 2
j= 1623 type= 2
j= 1624 type= 2
j= 1625 type= 3
j= 1626 type= 4
j= 1627 type= 3
j= 1628 type= 3
j= 1629 type= 1
j= 1630 type= 3
j= 1631 type= 4
j= 1632 type= 1
j= 1633 type= 2
j= 1634 type= 2
j= 1635 type= 2
j= 1636 type= 2
j= 1637 type= 2
j= 1638 type= 4
j= 1639 type= 4
j= 1640 type= 4
j= 1641 type= 4
j= 1642 type= 4
j= 1643 type= 1
j= 1644 type= 4
j= 1645 type= 4
j= 1646 type= 1
j= 1647 type= 3
j= 1648 type= 2
j= 1649 type= 3
j= 1650 type= 2
j= 1651 type= 3
j= 1652 type= 3
j= 1653 type= 2
j= 1654 type= 1
j= 1655 type= 3
j= 1656 type= 2
j= 1657 type= 3
j= 1658 type= 3
j= 1659 type= 1
j= 1660 type= 2
j= 1661 type= 2
j= 1662 type= 3
j= 1663 type= 3
j= 1664 type= 2
j= 1665 type= 3
j= 1666 type= 3
j= 1667 type= 1
j= 1668 type= 2
j= 1669 type= 2
j= 1670 type= 4
j= 1671 type= 3
j= 1672 type= 3
j= 1673 type= 3
j= 1674 type= 2
j= 1675 

j= 2127 type= 1
j= 2128 type= 2
j= 2129 type= 2
j= 2130 type= 2
j= 2131 type= 3
j= 2132 type= 3
j= 2133 type= 3
j= 2134 type= 3
j= 2135 type= 2
j= 2136 type= 3
j= 2137 type= 3
j= 2138 type= 2
j= 2139 type= 1
j= 2140 type= 2
j= 2141 type= 2
j= 2142 type= 2
j= 2143 type= 2
j= 2144 type= 4
j= 2145 type= 4
j= 2146 type= 4
j= 2147 type= 4
j= 2148 type= 4
j= 2149 type= 2
j= 2150 type= 4
j= 2151 type= 4
j= 2152 type= 2
j= 2153 type= 1
j= 2154 type= 2
j= 2155 type= 2
j= 2156 type= 2
j= 2157 type= 4
j= 2158 type= 4
j= 2159 type= 4
j= 2160 type= 4
j= 2161 type= 2
j= 2162 type= 3
j= 2163 type= 4
j= 2164 type= 3
j= 2165 type= 2
j= 2166 type= 2
j= 2167 type= 3
j= 2168 type= 3
j= 2169 type= 4
j= 2170 type= 3
j= 2171 type= 2
j= 2172 type= 2
j= 2173 type= 3
j= 2174 type= 3
j= 2175 type= 3
j= 2176 type= 2
j= 2177 type= 4
j= 2178 type= 3
j= 2179 type= 2
j= 2180 type= 2
j= 2181 type= 3
j= 2182 type= 3
j= 2183 type= 3
j= 2184 type= 3
j= 2185 type= 2
j= 2186 type= 2
j= 2187 type= 1
j= 2188 type= 3
j= 2189 

j= 2642 type= 4
j= 2643 type= 4
j= 2644 type= 4
j= 2645 type= 4
j= 2646 type= 1
j= 2647 type= 3
j= 2648 type= 2
j= 2649 type= 3
j= 2650 type= 2
j= 2651 type= 3
j= 2652 type= 3
j= 2653 type= 2
j= 2654 type= 2
j= 2655 type= 3
j= 2656 type= 2
j= 2657 type= 3
j= 2658 type= 3
j= 2659 type= 2
j= 2660 type= 2
j= 2661 type= 2
j= 2662 type= 3
j= 2663 type= 3
j= 2664 type= 2
j= 2665 type= 3
j= 2666 type= 3
j= 2667 type= 2
j= 2668 type= 2
j= 2669 type= 2
j= 2670 type= 2
j= 2671 type= 3
j= 2672 type= 3
j= 2673 type= 3
j= 2674 type= 2
j= 2675 type= 3
j= 2676 type= 3
j= 2677 type= 2
j= 2678 type= 2
j= 2679 type= 2
j= 2680 type= 2
j= 2681 type= 1
j= 2682 type= 4
j= 2683 type= 4
j= 2684 type= 4
j= 2685 type= 4
j= 2686 type= 2
j= 2687 type= 4
j= 2688 type= 4
j= 2689 type= 2
j= 2690 type= 2
j= 2691 type= 2
j= 2692 type= 2
j= 2693 type= 1
j= 2694 type= 2
j= 2695 type= 4
j= 2696 type= 4
j= 2697 type= 4
j= 2698 type= 4
j= 2699 type= 4
j= 2700 type= 2
j= 2701 type= 4
j= 2702 type= 4
j= 2703 type= 2
j= 2704 

j= 3155 type= 4
j= 3156 type= 4
j= 3157 type= 4
j= 3158 type= 2
j= 3159 type= 1
j= 3160 type= 2
j= 3161 type= 2
j= 3162 type= 4
j= 3163 type= 2
j= 3164 type= 1
j= 3165 type= 4
j= 3166 type= 4
j= 3167 type= 4
j= 3168 type= 4
j= 3169 type= 4
j= 3170 type= 4
j= 3171 type= 4
j= 3172 type= 4
j= 3173 type= 4
j= 3174 type= 4


# Define new level tags based on database

In [30]:
k=0
dblevels=[]
for i in range(nlevels):
    ii=i+1
    name1=levels.loc[ii]['CFG']
    ncfg1=nconfig(name1)
    s1=levels.loc[ii]['S']
    l1=levels.loc[ii]['L']
    g1=2*levels.loc[ii]['J']+1
    for j in range(nmodlev):
        name2=mod_levels.loc[j]['ElectronConfig']
        ncfg2=nconfig(name2)
        s2=mod_levels.loc[j]['S']
        l2=mod_levels.loc[j]['L']
        g2=mod_levels.loc[j]['LevelWeight']
        #if (ncfg1==ncfg2) & (ncfg1!=0) & (s1==s2) & (l1==l2) & (g1==g2): 
        if (ncfg1==ncfg2) & (s1==s2) & (l1==l2) & (g1==g2): 
            k=k+1
            # define variable dup:
            #     first element: AS index
            #     second element: DataBase index
            dup=[ii,j+1] 
            dblevels.append(dup)
            print("k=",k)
            print(ii,name1,ncfg1,s1,l1,g1)
            print(j+1,name2,ncfg2,s2,l2,g2)
            break

k= 1
1 3S2 1 1 0 1.0
1 2p6.3s2 1 1 0 1
k= 2
2 3S1 3P1 2 3 1 9.0
2 3s.3p 2 3 1 9
k= 3
3 3S1 3P1 2 1 1 3.0
3 3s.3p 2 1 1 3
k= 4
4 3S1 4S1 4 3 0 3.0
4 3s.4s 4 3 0 3
k= 5
5 3S1 4S1 4 1 0 1.0
5 3s.4s 4 1 0 1
k= 6
6 3S1 3D1 3 1 2 5.0
6 3s.3d 3 1 2 5
k= 7
7 3S1 4P1 5 3 1 9.0
7 3s.4p 5 3 1 9
k= 8
8 3S1 3D1 3 3 2 15.0
8 3s.3d 3 3 2 15
k= 9
9 3S1 4P1 5 1 1 3.0
9 3s.4p 5 1 1 3
k= 10
10 3S1 5S1 8 3 0 3.0
10 3s.5s 8 3 0 3
k= 11
11 3S1 5S1 8 1 0 1.0
11 3s.5s 8 1 0 1
k= 12
12 3S1 4D1 6 1 2 5.0
12 3s.4d 6 1 2 5
k= 13
13 3S1 4D1 6 3 2 15.0
13 3s.4d 6 3 2 15
k= 14
14 3S1 5P1 9 3 1 9.0
14 3s.5p 9 3 1 9
k= 15
15 3S1 4F1 7 3 3 21.0
16 3s.4f 7 3 3 21
k= 16
16 3S1 4F1 7 1 3 7.0
15 3s.4f 7 1 3 7
k= 17
17 3S1 5P1 9 1 1 3.0
17 3s.5p 9 1 1 3
k= 18
18 3S1 6S1 14 3 0 3.0
18 3s.6s 14 3 0 3
k= 19
19 3S1 6S1 14 1 0 1.0
19 3s.6s 14 1 0 1
k= 20
20 3S1 5D1 10 1 2 5.0
20 3s.5d 10 1 2 5
k= 21
21 3S1 5D1 10 3 2 15.0
21 3s.5d 10 3 2 15
k= 22
22 3S1 6P1 15 3 1 9.0
22 3s.6p 15 3 1 9
k= 23
23 3S1 5F1 11 3 3 21.0
24 3s.5f 11 3 

In [32]:
dblevels

[[1, 1],
 [2, 2],
 [3, 3],
 [4, 4],
 [5, 5],
 [6, 6],
 [7, 7],
 [8, 8],
 [9, 9],
 [10, 10],
 [11, 11],
 [12, 12],
 [13, 13],
 [14, 14],
 [15, 16],
 [16, 15],
 [17, 17],
 [18, 18],
 [19, 19],
 [20, 20],
 [21, 21],
 [22, 22],
 [23, 24],
 [24, 26],
 [25, 27],
 [26, 23],
 [27, 25],
 [28, 29],
 [29, 30],
 [30, 31],
 [31, 32],
 [32, 33],
 [33, 35],
 [34, 37],
 [35, 39],
 [36, 40],
 [37, 38],
 [38, 34],
 [39, 36],
 [40, 41],
 [41, 43],
 [42, 42],
 [43, 44],
 [44, 45],
 [45, 46],
 [46, 50],
 [47, 51],
 [48, 53],
 [49, 54],
 [50, 52],
 [51, 49],
 [52, 47],
 [53, 48],
 [54, 55],
 [65, 28]]

In [36]:
def printout(dblevels,ups):
    global ntran
    fp=open("Mg0+EffCollStr.dat","w+")
    zn=12
    ionch=0
    modidx=1
    source=999
    header=["AtomicNumber","IonCharge","ModelIndex","LowerLevel","UpperLevel",
             "Temperature","CollisionStrength","Source"]
    print(*header,sep="\t",file=fp)
    kk=0
    kn=0
    for j in range(ntran):
        k=int(ups.loc[j]["k"])
        i=int(ups.loc[j]["i"])
        print("j==",j)
        nk=0
        ni=0
        for ii in dblevels:
            dup=ii
            # lower state:
            if dup[0]==i: 
                ni=dup[1]
                print("i=",i,"AS=",dup[0]," ni=",dup[1])
                continue
            # upper state:
            if dup[0]==k: 
                nk=dup[1]
                print("k=",k,"AS=",dup[0]," nk=",dup[1])
                continue
        if (ni!=0) and (nk!=0):
            kk=kk+1
            print(ni,nk)
            for jj in range(ntemp):
                upsjj=ups.loc[j][jj+3]
                tjj=T[jj]
                if ni<nk: 
                    #print(zn,ionch,modidx,ni,nk,tjj,upsjj,source,sep="\t")
                    print(zn,ionch,modidx,ni,nk,tjj,upsjj,source,sep="\t",file=fp)
                if ni>nk: 
                    #print(zn,ionch,modidx,nk,ni,tjj,upsjj,source,sep="\t")
                    print(zn,ionch,modidx,nk,ni,tjj,upsjj,source,sep="\t",file=fp)
            if j==ntran-1: print("all print ok")
        else:
            kn=kn+1
            print("i or k not found")
    #    print("\n")
    print(kk," transitions found")
    print(kn," not transitions found")
    print(kk+kn," transitions total")
    return

In [38]:
printout(dblevels,ups)

j== 0
i= 1 AS= 1  ni= 1
k= 2 AS= 2  nk= 2
1 2
j== 1
i= 1 AS= 1  ni= 1
k= 3 AS= 3  nk= 3
1 3
j== 2
i= 2 AS= 2  ni= 2
k= 3 AS= 3  nk= 3
2 3
j== 3
i= 1 AS= 1  ni= 1
k= 4 AS= 4  nk= 4
1 4
j== 4
i= 2 AS= 2  ni= 2
k= 4 AS= 4  nk= 4
2 4
j== 5
i= 3 AS= 3  ni= 3
k= 4 AS= 4  nk= 4
3 4
j== 6
i= 1 AS= 1  ni= 1
k= 5 AS= 5  nk= 5
1 5
j== 7
i= 2 AS= 2  ni= 2
k= 5 AS= 5  nk= 5
2 5
j== 8
i= 3 AS= 3  ni= 3
k= 5 AS= 5  nk= 5
3 5
j== 9
i= 4 AS= 4  ni= 4
k= 5 AS= 5  nk= 5
4 5
j== 10
i= 1 AS= 1  ni= 1
k= 6 AS= 6  nk= 6
1 6
j== 11
i= 2 AS= 2  ni= 2
k= 6 AS= 6  nk= 6
2 6
j== 12
i= 3 AS= 3  ni= 3
k= 6 AS= 6  nk= 6
3 6
j== 13
i= 4 AS= 4  ni= 4
k= 6 AS= 6  nk= 6
4 6
j== 14
i= 5 AS= 5  ni= 5
k= 6 AS= 6  nk= 6
5 6
j== 15
i= 1 AS= 1  ni= 1
k= 7 AS= 7  nk= 7
1 7
j== 16
i= 2 AS= 2  ni= 2
k= 7 AS= 7  nk= 7
2 7
j== 17
i= 3 AS= 3  ni= 3
k= 7 AS= 7  nk= 7
3 7
j== 18
i= 4 AS= 4  ni= 4
k= 7 AS= 7  nk= 7
4 7
j== 19
i= 5 AS= 5  ni= 5
k= 7 AS= 7  nk= 7
5 7
j== 20
i= 6 AS= 6  ni= 6
k= 7 AS= 7  nk= 7
6 7
j== 21
i= 1 AS= 1  ni= 

k= 20 AS= 20  nk= 20
2 20
j== 173
i= 3 AS= 3  ni= 3
k= 20 AS= 20  nk= 20
3 20
j== 174
i= 4 AS= 4  ni= 4
k= 20 AS= 20  nk= 20
4 20
j== 175
i= 5 AS= 5  ni= 5
k= 20 AS= 20  nk= 20
5 20
j== 176
i= 6 AS= 6  ni= 6
k= 20 AS= 20  nk= 20
6 20
j== 177
i= 7 AS= 7  ni= 7
k= 20 AS= 20  nk= 20
7 20
j== 178
i= 8 AS= 8  ni= 8
k= 20 AS= 20  nk= 20
8 20
j== 179
i= 9 AS= 9  ni= 9
k= 20 AS= 20  nk= 20
9 20
j== 180
i= 10 AS= 10  ni= 10
k= 20 AS= 20  nk= 20
10 20
j== 181
i= 11 AS= 11  ni= 11
k= 20 AS= 20  nk= 20
11 20
j== 182
i= 12 AS= 12  ni= 12
k= 20 AS= 20  nk= 20
12 20
j== 183
i= 13 AS= 13  ni= 13
k= 20 AS= 20  nk= 20
13 20
j== 184
i= 14 AS= 14  ni= 14
k= 20 AS= 20  nk= 20
14 20
j== 185
i= 15 AS= 15  ni= 16
k= 20 AS= 20  nk= 20
16 20
j== 186
i= 16 AS= 16  ni= 15
k= 20 AS= 20  nk= 20
15 20
j== 187
i= 17 AS= 17  ni= 17
k= 20 AS= 20  nk= 20
17 20
j== 188
i= 18 AS= 18  ni= 18
k= 20 AS= 20  nk= 20
18 20
j== 189
i= 19 AS= 19  ni= 19
k= 20 AS= 20  nk= 20
19 20
j== 190
i= 1 AS= 1  ni= 1
k= 21 AS= 21  nk= 21
1 2

k= 27 AS= 27  nk= 25
5 25
j== 329
i= 6 AS= 6  ni= 6
k= 27 AS= 27  nk= 25
6 25
j== 330
i= 7 AS= 7  ni= 7
k= 27 AS= 27  nk= 25
7 25
j== 331
i= 8 AS= 8  ni= 8
k= 27 AS= 27  nk= 25
8 25
j== 332
i= 9 AS= 9  ni= 9
k= 27 AS= 27  nk= 25
9 25
j== 333
i= 10 AS= 10  ni= 10
k= 27 AS= 27  nk= 25
10 25
j== 334
i= 11 AS= 11  ni= 11
k= 27 AS= 27  nk= 25
11 25
j== 335
i= 12 AS= 12  ni= 12
k= 27 AS= 27  nk= 25
12 25
j== 336
i= 13 AS= 13  ni= 13
k= 27 AS= 27  nk= 25
13 25
j== 337
i= 14 AS= 14  ni= 14
k= 27 AS= 27  nk= 25
14 25
j== 338
i= 15 AS= 15  ni= 16
k= 27 AS= 27  nk= 25
16 25
j== 339
i= 16 AS= 16  ni= 15
k= 27 AS= 27  nk= 25
15 25
j== 340
i= 17 AS= 17  ni= 17
k= 27 AS= 27  nk= 25
17 25
j== 341
i= 18 AS= 18  ni= 18
k= 27 AS= 27  nk= 25
18 25
j== 342
i= 19 AS= 19  ni= 19
k= 27 AS= 27  nk= 25
19 25
j== 343
i= 20 AS= 20  ni= 20
k= 27 AS= 27  nk= 25
20 25
j== 344
i= 21 AS= 21  ni= 21
k= 27 AS= 27  nk= 25
21 25
j== 345
i= 22 AS= 22  ni= 22
k= 27 AS= 27  nk= 25
22 25
j== 346
i= 23 AS= 23  ni= 24
k= 27 AS=

k= 32 AS= 32  nk= 33
21 33
j== 485
i= 22 AS= 22  ni= 22
k= 32 AS= 32  nk= 33
22 33
j== 486
i= 23 AS= 23  ni= 24
k= 32 AS= 32  nk= 33
24 33
j== 487
i= 24 AS= 24  ni= 26
k= 32 AS= 32  nk= 33
26 33
j== 488
i= 25 AS= 25  ni= 27
k= 32 AS= 32  nk= 33
27 33
j== 489
i= 26 AS= 26  ni= 23
k= 32 AS= 32  nk= 33
23 33
j== 490
i= 27 AS= 27  ni= 25
k= 32 AS= 32  nk= 33
25 33
j== 491
i= 28 AS= 28  ni= 29
k= 32 AS= 32  nk= 33
29 33
j== 492
i= 29 AS= 29  ni= 30
k= 32 AS= 32  nk= 33
30 33
j== 493
i= 30 AS= 30  ni= 31
k= 32 AS= 32  nk= 33
31 33
j== 494
i= 31 AS= 31  ni= 32
k= 32 AS= 32  nk= 33
32 33
j== 495
i= 1 AS= 1  ni= 1
k= 33 AS= 33  nk= 35
1 35
j== 496
i= 2 AS= 2  ni= 2
k= 33 AS= 33  nk= 35
2 35
j== 497
i= 3 AS= 3  ni= 3
k= 33 AS= 33  nk= 35
3 35
j== 498
i= 4 AS= 4  ni= 4
k= 33 AS= 33  nk= 35
4 35
j== 499
i= 5 AS= 5  ni= 5
k= 33 AS= 33  nk= 35
5 35
j== 500
i= 6 AS= 6  ni= 6
k= 33 AS= 33  nk= 35
6 35
j== 501
i= 7 AS= 7  ni= 7
k= 33 AS= 33  nk= 35
7 35
j== 502
i= 8 AS= 8  ni= 8
k= 33 AS= 33  nk= 35
8 

k= 37 AS= 37  nk= 38
12 38
j== 641
i= 13 AS= 13  ni= 13
k= 37 AS= 37  nk= 38
13 38
j== 642
i= 14 AS= 14  ni= 14
k= 37 AS= 37  nk= 38
14 38
j== 643
i= 15 AS= 15  ni= 16
k= 37 AS= 37  nk= 38
16 38
j== 644
i= 16 AS= 16  ni= 15
k= 37 AS= 37  nk= 38
15 38
j== 645
i= 17 AS= 17  ni= 17
k= 37 AS= 37  nk= 38
17 38
j== 646
i= 18 AS= 18  ni= 18
k= 37 AS= 37  nk= 38
18 38
j== 647
i= 19 AS= 19  ni= 19
k= 37 AS= 37  nk= 38
19 38
j== 648
i= 20 AS= 20  ni= 20
k= 37 AS= 37  nk= 38
20 38
j== 649
i= 21 AS= 21  ni= 21
k= 37 AS= 37  nk= 38
21 38
j== 650
i= 22 AS= 22  ni= 22
k= 37 AS= 37  nk= 38
22 38
j== 651
i= 23 AS= 23  ni= 24
k= 37 AS= 37  nk= 38
24 38
j== 652
i= 24 AS= 24  ni= 26
k= 37 AS= 37  nk= 38
26 38
j== 653
i= 25 AS= 25  ni= 27
k= 37 AS= 37  nk= 38
27 38
j== 654
i= 26 AS= 26  ni= 23
k= 37 AS= 37  nk= 38
23 38
j== 655
i= 27 AS= 27  ni= 25
k= 37 AS= 37  nk= 38
25 38
j== 656
i= 28 AS= 28  ni= 29
k= 37 AS= 37  nk= 38
29 38
j== 657
i= 29 AS= 29  ni= 30
k= 37 AS= 37  nk= 38
30 38
j== 658
i= 30 AS= 30 

...

i= 2 AS= 2  ni= 2
i or k not found
j== 2230
i= 3 AS= 3  ni= 3
i or k not found
j== 2231
i= 4 AS= 4  ni= 4
i or k not found
j== 2232
i= 5 AS= 5  ni= 5
i or k not found
j== 2233
i= 6 AS= 6  ni= 6
i or k not found
j== 2234
i= 7 AS= 7  ni= 7
i or k not found
j== 2235
i= 8 AS= 8  ni= 8
i or k not found
j== 2236
i= 9 AS= 9  ni= 9
i or k not found
j== 2237
i= 10 AS= 10  ni= 10
i or k not found
j== 2238
i= 11 AS= 11  ni= 11
i or k not found
j== 2239
i= 12 AS= 12  ni= 12
i or k not found
j== 2240
i= 13 AS= 13  ni= 13
i or k not found
j== 2241
i= 14 AS= 14  ni= 14
i or k not found
j== 2242
i= 15 AS= 15  ni= 16
i or k not found
j== 2243
i= 16 AS= 16  ni= 15
i or k not found
j== 2244
i= 17 AS= 17  ni= 17
i or k not found
j== 2245
i= 18 AS= 18  ni= 18
i or k not found
j== 2246
i= 19 AS= 19  ni= 19
i or k not found
j== 2247
i= 20 AS= 20  ni= 20
i or k not found
j== 2248
i= 21 AS= 21  ni= 21
i or k not found
j== 2249
i= 22 AS= 22  ni= 22
i or k not found
j== 2250
i= 23 AS= 23  ni= 24
i or k not found

i or k not found
j== 2480
i= 47 AS= 47  ni= 51
i or k not found
j== 2481
i= 48 AS= 48  ni= 53
i or k not found
j== 2482
i= 49 AS= 49  ni= 54
i or k not found
j== 2483
i= 50 AS= 50  ni= 52
i or k not found
j== 2484
i= 51 AS= 51  ni= 49
i or k not found
j== 2485
i= 52 AS= 52  ni= 47
i or k not found
j== 2486
i= 53 AS= 53  ni= 48
i or k not found
j== 2487
i= 54 AS= 54  ni= 55
i or k not found
j== 2488
i or k not found
j== 2489
i or k not found
j== 2490
i or k not found
j== 2491
i or k not found
j== 2492
i or k not found
j== 2493
i or k not found
j== 2494
i or k not found
j== 2495
i or k not found
j== 2496
i or k not found
j== 2497
i or k not found
j== 2498
i or k not found
j== 2499
i or k not found
j== 2500
i or k not found
j== 2501
i or k not found
j== 2502
i or k not found
j== 2503
i or k not found
j== 2504
i= 1 AS= 1  ni= 1
i or k not found
j== 2505
i= 2 AS= 2  ni= 2
i or k not found
j== 2506
i= 3 AS= 3  ni= 3
i or k not found
j== 2507
i= 4 AS= 4  ni= 4
i or k not found
j== 2508
i= 5 A

i= 34 AS= 34  ni= 37
i or k not found
j== 2681
i= 35 AS= 35  ni= 39
i or k not found
j== 2682
i= 36 AS= 36  ni= 40
i or k not found
j== 2683
i= 37 AS= 37  ni= 38
i or k not found
j== 2684
i= 38 AS= 38  ni= 34
i or k not found
j== 2685
i= 39 AS= 39  ni= 36
i or k not found
j== 2686
i= 40 AS= 40  ni= 41
i or k not found
j== 2687
i= 41 AS= 41  ni= 43
i or k not found
j== 2688
i= 42 AS= 42  ni= 42
i or k not found
j== 2689
i= 43 AS= 43  ni= 44
i or k not found
j== 2690
i= 44 AS= 44  ni= 45
i or k not found
j== 2691
i= 45 AS= 45  ni= 46
i or k not found
j== 2692
i= 46 AS= 46  ni= 50
i or k not found
j== 2693
i= 47 AS= 47  ni= 51
i or k not found
j== 2694
i= 48 AS= 48  ni= 53
i or k not found
j== 2695
i= 49 AS= 49  ni= 54
i or k not found
j== 2696
i= 50 AS= 50  ni= 52
i or k not found
j== 2697
i= 51 AS= 51  ni= 49
i or k not found
j== 2698
i= 52 AS= 52  ni= 47
i or k not found
j== 2699
i= 53 AS= 53  ni= 48
i or k not found
j== 2700
i= 54 AS= 54  ni= 55
i or k not found
j== 2701
i or k not fo

j== 2887
i= 19 AS= 19  ni= 19
i or k not found
j== 2888
i= 20 AS= 20  ni= 20
i or k not found
j== 2889
i= 21 AS= 21  ni= 21
i or k not found
j== 2890
i= 22 AS= 22  ni= 22
i or k not found
j== 2891
i= 23 AS= 23  ni= 24
i or k not found
j== 2892
i= 24 AS= 24  ni= 26
i or k not found
j== 2893
i= 25 AS= 25  ni= 27
i or k not found
j== 2894
i= 26 AS= 26  ni= 23
i or k not found
j== 2895
i= 27 AS= 27  ni= 25
i or k not found
j== 2896
i= 28 AS= 28  ni= 29
i or k not found
j== 2897
i= 29 AS= 29  ni= 30
i or k not found
j== 2898
i= 30 AS= 30  ni= 31
i or k not found
j== 2899
i= 31 AS= 31  ni= 32
i or k not found
j== 2900
i= 32 AS= 32  ni= 33
i or k not found
j== 2901
i= 33 AS= 33  ni= 35
i or k not found
j== 2902
i= 34 AS= 34  ni= 37
i or k not found
j== 2903
i= 35 AS= 35  ni= 39
i or k not found
j== 2904
i= 36 AS= 36  ni= 40
i or k not found
j== 2905
i= 37 AS= 37  ni= 38
i or k not found
j== 2906
i= 38 AS= 38  ni= 34
i or k not found
j== 2907
i= 39 AS= 39  ni= 36
i or k not found
j== 2908
i= 4

j== 3157
i or k not found
j== 3158
i or k not found
j== 3159
i or k not found
j== 3160
i or k not found
j== 3161
i or k not found
j== 3162
i or k not found
j== 3163
i or k not found
j== 3164
i or k not found
j== 3165
i or k not found
j== 3166
i or k not found
j== 3167
i or k not found
j== 3168
i or k not found
j== 3169
i or k not found
j== 3170
i or k not found
j== 3171
i or k not found
j== 3172
i or k not found
j== 3173
i or k not found
j== 3174
i or k not found
1446  transitions found
1729  not transitions found
3175  transitions total
